In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/data0/pretrained-models/paraphrase-MiniLM-L6-v2', device='cuda:4')

/home/llong/anaconda3/envs/gxj/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
SPIDER_CSV_PATH='/data0/datasets/spider/database_csv'
SPIDER_EMB_PATH = '/data0/gxj/db_emb'
import os, torch
import numpy as np
import pandas as pd

for db_id in os.listdir(SPIDER_CSV_PATH):
    db_path = os.path.join(SPIDER_CSV_PATH, db_id)
    emb_path = os.path.join(SPIDER_EMB_PATH, db_id)
    if not os.path.exists(emb_path):
        os.makedirs(emb_path)
    for table_id in os.listdir(db_path):
        print(db_id, table_id)
        if not table_id.endswith('.csv'):
            continue
        
        torch.manual_seed(11)
        np.random.seed(11)
        row_size = 50
        column_size = 50

        embedding_dim = model.get_sentence_embedding_dimension()
        table_df = pd.read_csv(os.path.join(SPIDER_CSV_PATH, db_id, table_id), encoding='utf-8', low_memory=False)

        if len(table_df) > row_size:
            table_df = table_df.sample(n=row_size)
        table = table_df.to_numpy()
        table = table.astype(str)
        table_emb = np.zeros((table.shape[0], table.shape[1], embedding_dim))
        for j, row in enumerate(table):
            row_emb = model.encode(row)
            table_emb[j] = row_emb
        
        column_truncation = np.random.permutation(range(table_emb.shape[1]))[:column_size]
        table_emb = table_emb[:, column_truncation, :]

        table_emb_padded = np.zeros((row_size, 100, table_emb.shape[2]))
        table_emb_padded[:table_emb.shape[0], :table_emb.shape[1], :] = table_emb
        table_mask = np.zeros((row_size, 100))
        table_mask[:table_emb.shape[0], :table_emb.shape[1]] = 1

        
        output_dir = os.path.join(SPIDER_EMB_PATH, db_id, table_id.replace('.csv', '.pt'))
        # save (table_emb_padded, table_mask) to output_dir
        torch.save((table_emb_padded, table_mask), output_dir)



KeyboardInterrupt: 